In [1]:
!which python

/home/mostendorff/miniconda2/envs/legal-docsim/bin/python


In [2]:
!pip freeze|grep transformers

sentence-transformers==0.4.1.2
transformers==4.2.0


In [3]:
import re
import json
import os
import requests
import pickle
import pandas as pd
import logging
from pathlib import Path
from collections import defaultdict
from docsim.gold_standard import GoldStandard
from smart_open import open
import random
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from IPython.core.display import display

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from gensim.models import KeyedVectors

from docsim.methods import RecSys
from docsim.methods.keyed_vector_based import KeyedVectorRecSys, MultiKeyedVectorRecSys, EnsembleKeyedVectorRecSys
from docsim.methods.text_based import TfIdfRecSys, Doc2VecRecSys, WeightedAvgWordVectorsRecSys
from docsim.methods.transformer_based import TransformerRecSys
from docsim.methods.graph_based import GraphEmbeddingRecSys
from docsim.environment import get_env
from docsim.experiment import Experiment
from utils import get_mean_avg_precision, get_avg_precision, highlight_max


logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)



unable to import 'smart_open.gcs', disabling that module


In [4]:
!nvidia-smi

Fri Feb  5 19:25:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.31       Driver Version: 440.31       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1B:00.0 Off |                  N/A |
| 27%   33C    P8    12W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:1C:00.0 Off |                  Off |
| 33%   29C    P8     4W / 260W |  22331MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Q

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
#os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3,5,6,7'

In [6]:
from sentence_transformers.models import Pooling, Transformer
from smart_open import open
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, losses

In [7]:
train_batch_size = 13

data_dir = './data'
experiment = 'ocb'

models_dir = '/data/datasets/BERT_pre_trained_models/pytorch'

#model_name = 'legal-bert'
#model_name = 'longformer-large-4096'
model_name = 'legal-bert'

if 'long' in model_name:
    train_batch_size = 5

max_token_length = 512
train_epochs = 3

model_output_dir = f'./models/{experiment}/sentence_poincare3_{model_name}'

model_name_or_path = os.path.join(models_dir, model_name)
model_name_or_path

'/data/datasets/BERT_pre_trained_models/pytorch/legal-bert'

In [8]:
model_output_dir

'./models/ocb/sentence_poincare3_legal-bert'

In [9]:
from gensim.models import KeyedVectors

doc_model = KeyedVectors.load_word2vec_format('./models/ocb/poincare.w2v.txt')
doc_model.init_sims(replace=True)

INFO:gensim.models.utils_any2vec:loading projection weights from ./models/ocb/poincare.w2v.txt
INFO:gensim.models.utils_any2vec:loaded (1590, 300) matrix from ./models/ocb/poincare.w2v.txt
INFO:gensim.models.keyedvectors:precomputing L2-norms of word weight vectors


In [10]:
from pathlib import Path
from gensim.models import KeyedVectors

from docsim.environment import get_env
from docsim.experiment import Experiment
from docsim.methods.graph_based import GraphEmbeddingRecSys
from docsim.methods.text_based import TfIdfRecSys, Doc2VecRecSys, WeightedAvgWordVectorsRecSys
from docsim.methods.transformer_based import TransformerRecSys, SentenceTransformerRecSys


env = get_env()
data_dir = Path(data_dir)

logger.info(f'Experiment: {experiment}')

exp = Experiment(name=experiment, env=env, data_dir=data_dir)

exp.load_data()
exp.filter_docs()

INFO:__main__:Experiment: ocb


/data/experiments/mostendorff/legal-docsim/environments
Environment detected: gpu_server2 (in default.yml)


INFO:docsim.experiment:Documents loaded: 1,632
INFO:docsim.experiment:Unique documents in gold standard: 1,623
INFO:docsim.experiment:Documents after filtering: 1,590 (before 1,632)


In [11]:
a_ids = []
a_texts = []
b_ids = []
b_texts = []
labels = []
pairs = set()

for a_id in exp.doc_id2idx:
    for b_id in exp.doc_id2idx:
        if a_id != b_id:
            pair = tuple(sorted([a_id, b_id]))
            
            if pair not in pairs:

                #a_ids.append(a_id)
                #b_ids.append(b_id)

                a_texts.append(exp.texts[exp.doc_id2idx[a_id]])
                b_texts.append(exp.texts[exp.doc_id2idx[b_id]])
                
                sim = doc_model.similarity(a_id, b_id)
                
                labels.append(sim)
                pairs.add(pair)
                
len(labels)

1263255

In [12]:
# sub sample
samples = random.sample(list(zip(a_texts, b_texts, labels)), 100_000)


In [13]:
from sentence_transformers import InputExample

# min(1., max(.0, label))
train_examples = [InputExample(texts=[a, b], label=label) for a, b, label in samples]

In [14]:
# Sentence transformer
word_embedding_model = Transformer(model_name_or_path, max_seq_length=max_token_length)
pooling_model = Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [15]:
import torch

from transformers import BertTokenizerFast

#tokenizer = BertTokenizerFast.from_pretrained(model_name_or_path)
tokenizer = None

In [16]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)

In [17]:
train_loss = losses.CosineSimilarityLoss(model)

In [18]:
gs_df = exp.gs.df.sample(n=100)
gs_df

,seed_id,target_id,label
91905,4609,11182643,/casebooks/360-constitutional-law-separation-o...
45378,1400301,519963,/casebooks/715-contracts-cases-and-materials/
105043,3712729,5991514,"/casebooks/368-karlan-torts-materials-part-i/,..."
100632,1204707,1769262,/casebooks/510-open-source-property/
38025,4214321,10175054,/casebooks/510-open-source-property/
...,...,...,...
22595,1502994,8690012,"/casebooks/303-contracts/,/casebooks/477-contr..."
100416,6037278,7792231,"/casebooks/367-criminal-law-spring-2016/,/case..."
88123,6192512,3688385,/casebooks/441-summer-2016-first-amendment-rea...
6404,2304308,4376899,"/casebooks/303-contracts/,/casebooks/477-contr..."


In [19]:

from_texts = []
to_texts = []

for from_id, to_id in gs_df[['seed_id', 'target_id']].values:
    if from_id in exp.doc_id2idx and to_id in exp.doc_id2idx:
        from_texts.append(exp.texts[exp.doc_id2idx[from_id]])
        to_texts.append(exp.texts[exp.doc_id2idx[to_id]])
        
labels = [1] * len(from_texts)



In [20]:
from_texts[:3]

["Strauder v. West Virginia\nStrauder v. West Virginia.\n1. The Fourteenth Amendment of the Constitution of the United States considered, and held to be one of a series of constitutional provisions having a common purpose; namely, to secure to a recently emancipated race, which had been held in slavery through many generations, all the civil rights that the. superior race enjoy, and to give to it the protection of the general government, in the enjoyment of such rights, whenever they should be denied by the States. Whether the amendment had other, and if so what, purposes, not decided.\n2. The amendment not only gave citizenship and the privileges of citizenship to persons of color, but denied to any State the power to withhold from them the equal protection of the laws, and invested Congress with power, by appropriate legislation, to enforce its provisions.\n3. The amendment, although prohibitory in terms, confers by necessary implication a positive immunity, or right, most valuable t

In [21]:
from sentence_transformers.evaluation.BinaryClassificationEvaluator import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(from_texts, to_texts, labels)

In [ ]:
# Training
model.fit(
    train_objectives=[(train_dl, train_loss)],
    epochs=train_epochs, # try 1-4
    warmup_steps=100,
    #evaluator=evaluator,
    #evaluation_steps=evaluation_steps,  # increase to 5000 (full dataset => 20k steps)
    output_path=model_output_dir,
    #output_path_ignore_not_empty=True
)


In [ ]:
model.save(model_output_dir)

In [25]:
model_output_dir

'./models/ocb/sentence_poincare3_legal-bert'

In [26]:
model_output_dir

'./models/ocb/sentence_poincare3_legal-bert'